### <font color='turchesa'>Objetivo do Projeto: Prever Múltiplos Indicadores Macroeconômicos com Regressão Multi-Target</font>

Link do Dataset: https://www.kaggle.com/datasets/thomasfrewtwo/usmacros

### <font color='turchesa'>Instalando e Carregando Pacotes</font>

In [1]:
# Imports
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

### <font color='turchesa'>Carregando os Dados</font>

In [2]:
# Carregando df
df = pd.read_csv('macroeconomics.csv')

In [3]:
# Opção para mostrar todas as colunas do df
pd.set_option('display.max_columns', None)

In [4]:
df.head()

,date,Gross Domestic Product,Gross National Product,Gross domestic product per capita,Sticky Price Consumer Price Index less Food and Energy,Producer Price Index by Commodity: All Commodities,Unemployment Rate,Labor Force Participation Rate,Personal Consumption Expenditures,Net domestic investment: Private: Domestic business,Federal Government: Current Expenditures,"Trade Balance: Goods and Services, Balance of Payments Basis",Balance on current account,U.S. Dollars to Euro Spot Exchange Rate,Japanese Yen to U.S. Dollar Spot Exchange Rate,Federal Funds Effective Rate,M1,M2,Federal Debt: Total Public Debt,Federal Surplus or Deficit [-],Advance Retail Sales: Retail Trade,Industrial Production: Total Index,New Privately-Owned Housing Units Started: Total Units,Total Construction Spending: Total Construction in the United States,Consumer Opinion Surveys: Confidence Indicators: Composite Indicators: OECD Indicator for United States,Business Tendency Surveys (Manufacturing): Confidence Indicators: Composite Indicators: OECD Indicator for United States,Leading Index for the United States,Retail Inventories: Clothing and Clothing Accessory Stores,Total Business Inventories,5-Year High Quality Market (HQM) Corporate Bond Par Yield,Corporate Profits After Tax (without IVA and CCAdj),Real Exports of Goods and Services,"Rest of the World; Foreign Direct Investment in U.S.; Asset (Current Cost), Transactions",Equity Market Volatility Tracker: Exchange Rates,Producer Price Index by Commodity: Metals and Metal Products: Gold Ores,15-Year Fixed Rate Mortgage Average in the United States,Disposable Personal Income,CBOE Volatility Index: VIX,ICE BofA US High Yield Index Effective Yield,Employment Cost Index: Total compensation: All Civilian,Federal Debt: Total Public Debt as Percent of Gross Domestic Product,Personal Saving,Gross Domestic Product (Growth),Gross National Product (Growth),Gross domestic product per capita (Growth),Sticky Price Consumer Price Index less Food and Energy (Growth),Producer Price Index by Commodity: All Commodities (Growth),Unemployment Rate (Growth),Labor Force Participation Rate (Growth),Personal Consumption Expenditures (Growth),Net domestic investment: Private: Domestic business (Growth),Federal Government: Current Expenditures (Growth),"Trade Balance: Goods and Services, Balance of Payments Basis (Growth)",Balance on current account (Growth),U.S. Dollars to Euro Spot Exchange Rate (Growth),Japanese Yen to U.S. Dollar Spot Exchange Rate (Growth),Federal Funds Effective Rate (Growth),M1 (Growth),M2 (Growth),Federal Debt: Total Public Debt (Growth),Federal Surplus or Deficit [-] (Growth),Advance Retail Sales: Retail Trade (Growth),Industrial Production: Total Index (Growth),New Privately-Owned Housing Units Started: Total Units (Growth),Total Construction Spending: Total Construction in the United States (Growth),Consumer Opinion Surveys: Confidence Indicators: Composite Indicators: OECD Indicator for United States (Growth),Business Tendency Surveys (Manufacturing): Confidence Indicators: Composite Indicators: OECD Indicator for United States (Growth),Leading Index for the United States (Growth),Retail Inventories: Clothing and Clothing Accessory Stores.1,Total Business Inventories (Growth),5-Year High Quality Market (HQM) Corporate Bond Par Yield (Growth),Corporate Profits After Tax (without IVA and CCAdj) (Growth),Real Exports of Goods and Services (Growth),"Rest of the World; Foreign Direct Investment in U.S.; Asset (Current Cost), Transactions (Growth)",Equity Market Volatility Tracker: Exchange Rates (Growth),Producer Price Index by Commodity: Metals and Metal Products: Gold Ores (Growth),15-Year Fixed Rate Mortgage Average in the United States (Growth),Disposable Personal Income (Growth),CBOE Volatility Index: VIX (Growth),ICE BofA US High Yield Index Effective Yield (Growth),Employment Cost Index: Total compensation: All Civilian (Growth),Federal Debt: Total Public Debt as Percent of Gross Domestic Product (G

O objetivo do projeto será prever as variáveis GDP (Gross Domestic Product), GNP(Gross National Product) e Unemployment Rate (Taxa de Desemprego) com base em todos os demais indicadores. A fim de deixar o dataset mais visual, será alterado o nome de todas as demais variáveis para deixar o dataset mais visual e sem nomes de colunas extensos.

In [5]:
# Renomeando as colunas
df.rename(columns={'Gross Domestic Product': 'gdp', 
                   'Gross National Product': 'gnp', 
                   'Unemployment Rate': 'unemployment_rate'}, 
          inplace=True)

In [6]:
# Renomeando as colunas

# Selecionando colunas para renomear
cols_to_rename = [col for col in df.columns if col not in ['date', 'gdp', 'gnp', 'unemployment_rate']]

# Criando novos nomes
new_names = {col: f'v{i+1}' for i, col in enumerate(cols_to_rename)}

# Renomeando
df = df.rename(columns=new_names)

In [7]:
# Verificando as colunas renomeadas
df.columns

Index(['date', 'gdp', 'gnp', 'v1', 'v2', 'v3', 'unemployment_rate', 'v4', 'v5',
       'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16',
       'v17', 'v18', 'v19', 'v20', 'v21', 'v22', 'v23', 'v24', 'v25', 'v26',
       'v27', 'v28', 'v29', 'v30', 'v31', 'v32', 'v33', 'v34', 'v35', 'v36',
       'v37', 'v38', 'v39', 'v40', 'v41', 'v42', 'v43', 'v44', 'v45', 'v46',
       'v47', 'v48', 'v49', 'v50', 'v51', 'v52', 'v53', 'v54', 'v55', 'v56',
       'v57', 'v58', 'v59', 'v60', 'v61', 'v62', 'v63', 'v64', 'v65', 'v66',
       'v67', 'v68', 'v69', 'v70', 'v71', 'v72', 'v73', 'v74', 'v75', 'v76',
       'v77', 'v78', 'v79'],
      dtype='object')

In [8]:
# Verificando as últimas datas
df.date.tail(3)

85     4/1/2022
86     7/1/2022
87    10/1/2022
Name: date, dtype: object

É possível verificar no código acima que os indicadores são mostrados trimestralmente, e como a última data é de 01/10/2022, a previsão do nosso modelo será referente à 01/01/2023. A coluna date será removida pois não será usada no modelo para as previsões, porém é importante verificá-la antes para entendermos os períodos dos valores.

In [9]:
# Removendo coluna data
df.drop('date', axis = 1, inplace = True)

In [10]:
# Vamos mover a coluna unemployment_rate para melhor organização das variáveis alvo
df = pd.concat([df['unemployment_rate'], df[[col for col in df.columns if col != 'unemployment_rate']]], axis=1)

In [11]:
df.head()

,unemployment_rate,gdp,gnp,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79
0,4.2,10470.231,10510.297,36845,2.950054,137.767,67.2,6995.3,414.895,1982.626,-32438,-107136,0.9220,118.25,5.60,1101.9,5020.8,5773740,-7483.666667,251895,91.4115,1605,818728,100.791799,97.477603,0.69,36575,1191871,6.35,528.836,-4.9,236820,0.34586,81.700,6.59,7707.3,25.73,12.99,84.7,55.14434,392.3,0.33047,0.18938,0.10052,0.61014,1.64781,8.54701,0.39860,1.10861,-23.57829,2.26068,-2.21662,0.80637,6.19328,7.64946,-13.55839,0.58633,2.74453,1.96962,857.39872,0.46210,-1.23000,3.95078,0.34780,-1.25867,-1.05805,-30.06757,0.38608,-0.20205,-10.80955,7.52852,25.64103,-39.19648,36.70127,-1.04966,-9.66994,1.56330,-1.11550,-5.83002,0.00000,1.63375,11.72283
1,4.4,10599.000,10647.427,37211,3.132302,136.233,66.8,7042.2,384.743,2003.581,-28922,-95251,0.8736,122.62,4.33,1121.8,5146.9,5726815,64579.666670,254668,90.2516,1630,848756,100.627514,97.724029,0.43,36283,1179906,6.21,551.684,-11.2,238256,0.16107,81.200,6.66,7723.6,23.92,13.04,85.5,54.03165,361.7,1.22986,1.30472,0.99335,6.17781,-1.11299,3.93701,-0.59553,0.67093,-7.26738,1.05693,-10.83903,-11.09338,-5.24261,3.69039,-22.69603,1.80249,2.51295,-0.81273,-962.94152,1.10059,-1.26888,1.55763,3.66768,-0.16300,0.25280,-37.19807,-0.79926,-1.00386,-2.30850,4.32043,128.57143,0.60637,-53.42862,-0.61200,1.07401,0.21149,-7.00696,0.37295,0.94451,-2.01778,-7.80799
2,4.8,10598.020,10623.039,37110,3.293858,133.367,66.7,7070.3,336.012,2037.703,-29404,-103435,0.8908,121.63,3.50,1157.4,5240.9,5807464,-13889.666670,252467,89.0088,1600,845116,100.285004,98.315154,0.20,36274,1161962,5.82,505.610,-19.0,56512,0.25789,84.467,6.51,7896.1,25.38,13.10,86.4,54.79763,507.0,-0.00925,-0.22905,-0.27143,5.15773,-2.10423,9.84848,-0.09985,0.39902,-12.66586,1.70305,1.66653,8.59204,1.96428,-0.80656,-19.07183,3.17360,1.82648,1.40827,-121.50780,-0.86400,-1.37697,-1.86094,-0.42886,-0.34037,0.60489,-54.61538,-0.02572,-1.52080,-6.22986,-8.35152,69.64286,-76.28098,60.10720,4.02299,-2.29845,2.23298,6.08820,0.45458,1.05263,1.41765,40.18063
3,5.5,10660.465,10748.408,37234,3.560389,129.400,66.7,7187.3,229.850,2045.404,-29360,-88263,0.8951,123.74,2.13,1179.0,5384.5,5943439,-11785.333330,261687,88.0348,1570,846514,99.951708,98.202670,-0.26,36109,1127980,5.45,489.453,-13.2,140596,0.27196,85.267,6.25,7768.1,27.91,13.22,87.2,55.75216,270.2,0.58921,1.18016,0.33414,8.09177,-2.97426,13.79310,0.00000,1.65433,-31.59470,0.37793,-0.15191,-14.66815,0.48888,1.73812,-39.15689,1.87093,2.73964,2.34138,-15.15035,3.65196,-1.09431,-1.85455,0.16538,-0.33235,-0.11441,-230.50847,-0.45488,-2.92454,-6.30011,-3.19555,-30.52632,148.78964,5.45581,0.94712,-3.90117,-1.62106,9.96701,0.86613,0.92593,1.74192,-46.70304
4,5.7,10783.500,10833.143,37582,3.529122,128.900,66.6,7217.7,269.353,2091.469,-31113,-102608,0.8770,132.42,1.73,1186.5,5477.2,6006032,-32188.333330,257012,88.6851,1723,855181,100.796158,99.728727,0.58,35242,1112480,5.65,538.088,9.0,98976,0.50059,89.367,6.46,7966.4,21.36,12.41,87.9,55.69650,445.7,1.15412,0.78835,0.93463,-0.87819,-0.38640,3.63636,-0.09995,0.42343,17.18643,2.25212,5.97305,16.25256,-2.02269,7.01623,-18.66032,0.63694,1.72120,1.05314,173.12196,-1.78661,0.73872,9.74522,1.02389,0.84486,1.55399,-327.27273,-2.39924,-1.37417,3.60636,9.93660,-168.18182,-29.60255,84.06751,4.80844,3.33374,2.55276,-23.46531,-6.07268,0.80275,-0.09983,64.94388


### <font color='turchesa'>Separando Atributos e Targets</font>

In [12]:
# Separar atributos e targets
X = df[[col for col in df.columns if col not in ['unemployment_rate', 'gdp', 'gnp']]]
y = df[['unemployment_rate', 'gdp', 'gnp']]

In [13]:
# Dividir os dados em conjuntos de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.2, random_state = 42)

### <font color='turchesa'>Padronização dos Atributos</font>

In [14]:
# Padronizar as features
scaler = StandardScaler()
X_treino_scaled = scaler.fit_transform(X_treino)
X_teste_scaled = scaler.transform(X_teste)

Nos modelos de aprendizado de máquina com um único alvo (regressão ou classificação binária/multiclasse), a escala da variável alvo geralmente não afeta o treinamento do modelo. O modelo aprende a mapear as características de entrada para o valor do alvo, independentemente da escala deste. Por exemplo, prever o preço de uma casa (regressão) ou se um cliente irá comprar um produto (classificação) não requer a padronização do alvo.

No entanto, em modelos multi-alvo (multi-output), onde se prevêem múltiplas variáveis simultaneamente, a diferença de escala entre os alvos pode ser problemática. Se um alvo tem uma escala muito maior (ex: PIB em milhões) do que outro (ex: taxa de desemprego em porcentagem), o modelo pode ser dominado pelo alvo de maior escala, prejudicando a previsão dos alvos de menor escala. A padronização ou normalização dos alvos garante que todos eles contribuam igualmente para o treinamento do modelo, melhorando o desempenho geral, especialmente para os alvos com escalas menores.

In [15]:
# Ajustar o scaler APENAS aos dados de TREINO e transformar
y_treino = scaler.fit_transform(y_treino)

# Transformar os dados de TESTE (sem ajustar!)
y_teste = scaler.transform(y_teste)

### <font color='turchesa'>Construção do Modelo Multi-Target</font>

A Regressão Multi-Target, ou Regressão Multi-Saída, é uma abordagem usada para prever múltiplas variáveis dependentes simultaneamente. Em vez de prever uma única variável de saída, o modelo faz previsões para diversas variáveis de saída. Essa abordagem pode ser vantajosa quando há correlações entre as variáveis de saída, permitindo que o modelo capture essas dependências e forneça previsões mais precisas.

Existem várias maneiras de implementar a Regressão Multi-Target:

**Modelos Independentes**: Treinar um modelo de regressão separado para cada variável de saída. Isso é simples, mas ignora possíveis correlações entre as variáveis de saída.

**MultiOutputRegressor**: Utiliza um modelo de regressão base e estende para múltiplas saídas. Cada saída é tratada como uma regressão separada, mas o mesmo algoritmo de regressão é usado para todas as saídas. Por exemplo, MultiOutputRegressor(RandomForestRegressor(...)).

**Modelos Multivariados**: Alguns algoritmos de regressão são naturalmente capazes de lidar com múltiplas saídas, como Redes Neurais e alguns métodos de aprendizado profundo. Esses modelos podem capturar melhor as dependências entre as variáveis de saída.

**Modelos de Regressão Conjunta**: Utilizar uma abordagem que trata todas as variáveis de saída simultaneamente em um único modelo. Por exemplo, o algoritmo PLS (Partial Least Squares) pode ser usado para regressão multi-target.

In [16]:
# Construir e treinar o modelo
modelo = MultiOutputRegressor(RandomForestRegressor(n_estimators = 100, random_state = 42))

O código acima define um modelo de regressão multi-saída usando uma combinação do MultiOutputRegressor e do RandomForestRegressor.

**RandomForestRegressor(n_estimators = 100, random_state = 42)**: Este é o modelo de regressão base que será utilizado. Ele consiste em uma floresta de 100 árvores de decisão (indicadas por n_estimators = 100). O parâmetro random_state = 42 é usado para garantir a reprodutibilidade dos resultados, fixando a semente do gerador de números aleatórios.

**MultiOutputRegressor(...)**: Esta classe permite estender um modelo de regressão para lidar com múltiplas variáveis dependentes (multi-saída). Cada saída é tratada como uma regressão separada.

Portanto, modelo é um modelo que utiliza uma floresta aleatória para prever múltiplas variáveis dependentes simultaneamente.

### <font color='turchesa'>Treinamento e Avaliação do Modelo</font>

In [17]:
# Treinamento
modelo.fit(X_treino_scaled, y_treino)

MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

In [18]:
# Fazer previsões
y_pred = modelo.predict(X_teste_scaled)

In [19]:
# Mean Squared Error
# O multioutp = raw_values é usado pois há tres previsões para entregar
mse = mean_squared_error(y_teste, y_pred, multioutput = 'raw_values')
print("Mean Squared Error de cada variável alvo:", mse)

Mean Squared Error de cada variável alvo: [0.05934828 0.006243   0.00623429]


O Mean Squared Error (MSE) é uma métrica de avaliação usada para medir a diferença entre os valores previstos por um modelo e os valores reais observados.  O MSE calcula a média dos quadrados dos erros, ou seja, a média das diferenças quadráticas entre os valores reais e previstos. Um MSE menor indica um modelo mais preciso.

In [20]:
# Coeficiente de determinação
r2 = r2_score(y_teste, y_pred, multioutput = 'raw_values')
print("R2 de cada variável alvo:", r2)

R2 de cada variável alvo: [0.87316687 0.99601633 0.99602492]


O coeficiente de determinação, também conhecido como 𝑅2, é uma métrica que avalia a proporção da variabilidade da variável dependente que é explicada pelo modelo. Ele é amplamente usado para medir a qualidade do ajuste de modelos de regressão. 

O 𝑅2 varia entre 0 e 1, onde:

- 0 indica que o modelo não explica nenhuma variabilidade da variável alvo.
- 1 indica que o modelo explica toda a variabilidade da variável alvo.

In [21]:
# Inverter a transformação para obter as previsões na escala original
y_teste = scaler.inverse_transform(y_teste)

y_pred = scaler.inverse_transform(y_pred)

In [22]:
# Visualizar os resultados
resultados = pd.DataFrame(y_teste, columns=['unemployment_rate', 'gdp', 'gnp'])
resultados['unemployment_rate_pred'] = y_pred[:, 0]
resultados['gdp_pred'] = y_pred[:, 1]
resultados['gnp_pred'] = y_pred[:, 2]

In [23]:
round(resultados.head(),2)

,unemployment_rate,gdp,gnp,unemployment_rate_pred,gdp_pred,gnp_pred
0,3.8,21538.03,21794.02,4.62,21088.93,21487.20
1,4.2,10470.23,10510.30,5.22,10968.72,10962.54
2,4.7,14564.12,14681.75,4.74,14375.51,14483.54
3,4.6,13870.19,13906.65,4.74,13876.56,13834.16
4,5.7,11923.45,12021.46,5.86,11995.38,11971.43


In [24]:
round(resultados.tail(),2)

,unemployment_rate,gdp,gnp,unemployment_rate_pred,gdp_pred,gnp_pred
13,6.2,22313.85,22511.17,6.35,22655.49,22874.20
14,3.8,24740.48,24929.18,4.47,24184.70,24611.31
15,4.6,19148.19,19420.47,4.82,19237.02,19212.47
16,9.3,14381.24,14504.00,8.55,14497.17,14636.87
17,6.8,21704.71,21867.37,6.02,21744.23,22300.30


### <font color='turchesa'>Sistema e Versões dos Pacotes</font>

In [25]:
%reload_ext watermark
%watermark -v -m
%watermark --iversions

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.25.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 186 Stepping 3, GenuineIntel
CPU cores   : 12
Architecture: 64bit

matplotlib: 3.8.4
numpy     : 1.26.4
sklearn   : 1.4.2
seaborn   : 0.13.2
pandas    : 2.2.2

